<a href="https://colab.research.google.com/github/pete602d/Pythonslange/blob/main/udrykning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Udringskøretejer og reaktionstider
Her simuleres trafiksituationer med bilister og udrykningskøretøjer. Formålet er at visualisere hvor langt tid biler har til at reagere på udrykningskøretøjer. Her sammenlignes den nuværende situation, hvor bilister skal lytte efter sirener, med en ny metode hvor de bliver advaret om udrykningskøretøjet på forhånd.  


In [ ]:
import random
import pandas as pd

antal_biler = 20
uden_alarm = antal_biler//2
med_alarm = antal_biler//2

uden_alarm_reaktionstid = [random.randint(5, 25) for _ in range(uden_alarm)]
med_alarm_reaktionstid = [random.randint(15, 40) for _ in range(med_alarm)]

def forsinkelse_chance(reaktionstid, bilnummer, alarm_type):
  if reaktionstid > 20:
    chance = 0 #0%chance for forsinkelse
  elif reaktionstid < 10:
    chance = 100 #100%chance for forsinkelse
  else:
    chance = int((20 - reaktionstid) / 20 * 100)

#Simuleriung af forsinkelse
  if random.random() < chance /100:
     print(f"Bil {bilnummer} ({alarm_type}) har {reaktionstid} sekunder til at reagere og forsinker køretøjet")
  else:
     print(f"Bil {bilnummer} ({alarm_type}) har {reaktionstid} sekunder til at reagere og forsinker ikke køretøjet")

  return chance

forsinkelser_uden_alarm = []
for i, tid in enumerate(uden_alarm_reaktionstid):
    forsinkelser_uden_alarm.append(forsinkelse_chance(tid, i + 1, "uden alarm"))

forsinkelser_med_alarm = []
for i, tid in enumerate(med_alarm_reaktionstid):
    forsinkelser_med_alarm.append(forsinkelse_chance(tid, i + antal_biler // 2 + 1, "med alarm"))


#andel af fosinkelser for hver liste
andel_uden_alarm = sum(1 for chance in forsinkelser_uden_alarm if chance > 0) / len(forsinkelser_uden_alarm) * 100
andel_med_alarm = sum(1 for chance in forsinkelser_med_alarm if chance > 0) / len(forsinkelser_med_alarm) * 100

print("Andel af biler der forsinker uden alarm: {:.2f}%".format(andel_uden_alarm))

print("Andel af biler der forsinker med alarm: {:.2f}%".format(andel_med_alarm))


Bil 1 (uden alarm) har 9 sekunder til at reagere og forsinker køretøjet
Bil 2 (uden alarm) har 23 sekunder til at reagere og forsinker ikke køretøjet
Bil 3 (uden alarm) har 17 sekunder til at reagere og forsinker ikke køretøjet
Bil 4 (uden alarm) har 18 sekunder til at reagere og forsinker ikke køretøjet
Bil 5 (uden alarm) har 9 sekunder til at reagere og forsinker køretøjet
Bil 6 (uden alarm) har 25 sekunder til at reagere og forsinker ikke køretøjet
Bil 7 (uden alarm) har 12 sekunder til at reagere og forsinker ikke køretøjet
Bil 8 (uden alarm) har 7 sekunder til at reagere og forsinker køretøjet
Bil 9 (uden alarm) har 22 sekunder til at reagere og forsinker ikke køretøjet
Bil 10 (uden alarm) har 23 sekunder til at reagere og forsinker ikke køretøjet
Bil 11 (med alarm) har 16 sekunder til at reagere og forsinker ikke køretøjet
Bil 12 (med alarm) har 20 sekunder til at reagere og forsinker ikke køretøjet
Bil 13 (med alarm) har 19 sekunder til at reagere og forsinker ikke køretøjet
Bil